Install library cryptography: pip install cryptography

In [2]:
from cryptography.hazmat.primitives.ciphers import Cipher, algorithms, modes
from cryptography.hazmat.primitives import padding
from cryptography.hazmat.backends import default_backend
import os

In [5]:
# Fungsi untuk mengenkripsi teks
def encrypt(plaintext, key):
    iv = os.urandom(16)  # Generate IV (16 byte)
    cipher = Cipher(algorithms.AES(key), modes.CBC(iv), backend=default_backend())

    # Padding untuk memastikan ukuran blok 16 byte
    padder = padding.PKCS7(128).padder()
    padded_data = padder.update(plaintext.encode()) + padder.finalize()

    encryptor = cipher.encryptor()
    ciphertext = encryptor.update(padded_data) + encryptor.finalize()

    return iv + ciphertext  # Gabungkan IV dengan ciphertext

In [4]:
# Fungsi untuk mendekripsi teks
def decrypt(ciphertext, key):
    iv = ciphertext[:16]  # Ambil IV dari awal ciphertext
    encrypted_data = ciphertext[16:]

    cipher = Cipher(algorithms.AES(key), modes.CBC(iv), backend=default_backend())

    decryptor = cipher.decryptor()
    decrypted_padded = decryptor.update(encrypted_data) + decryptor.finalize()

    # Hapus padding
    unpadder = padding.PKCS7(128).unpadder()
    plaintext = unpadder.update(decrypted_padded) + unpadder.finalize()

    return plaintext.decode()

In [9]:
# Kunci AES 256-bit (32 byte)
key = os.urandom(32)

# Input teks yang akan dienkripsi
plaintext = "Ini pesan rahasia"

# Enkripsi
ciphertext = encrypt(plaintext, key)
print(f"Ciphertext: {ciphertext.hex()}")

# Dekripsi
decrypted_text = decrypt(ciphertext, key)
print(f"Decrypted Text: {decrypted_text}")

Ciphertext: 9b1b001d1834afe4132ba8468900fca3b6d3884b3b0e0ee6e2e65929a6bd4799d4fa2899e874f3900fa55a8eeb8ee678
Decrypted Text: Ini pesan rahasia


1. Jelaskan perbedaan peran algoritma AES,  mode CBC dan padding
2. Apakah yang digunakan kriptografi asimetris atau simetris jelaskan
3. Pada fungsi decrypt gantilah menjadi iv=os.urandom(16), apa yang terjadi jelaskan
4. Pada isi pesan plaintext, cobalah dari 1 huruf, 2 huruf dst. Lihatlah perubahan panjang chipertext, analisis perbedaannya
5. Bagaimana praktis pengamanan'key' agar isi pesan tidak mudah diketahui 

In [10]:
import time
# Fungsi enkripsi dengan berbagai mode
def encrypt_aes(plaintext, key, mode):
    iv = os.urandom(16)
    if mode == "CBC":
        cipher_mode = modes.CBC(iv)
    elif mode == "CFB":
        cipher_mode = modes.CFB(iv)
    elif mode == "GCM":
        cipher_mode = modes.GCM(iv)
    else:
        raise ValueError("Mode tidak didukung")

    cipher = Cipher(algorithms.AES(key), cipher_mode, backend=default_backend())
    plaintext_bytes = plaintext.encode()

    # Padding hanya diperlukan untuk CBC
    if mode == "CBC":
        padder = padding.PKCS7(128).padder()
        plaintext_bytes  = padder.update(plaintext_bytes)+ padder.finalize()

    encryptor = cipher.encryptor()
    ciphertext = encryptor.update(plaintext_bytes) + encryptor.finalize()

    return iv + ciphertext

# Uji performa mode AES
key = os.urandom(32)
plaintext = "Data rahasia untuk eksperimen yang lebih kompleks."

for mode in ["CBC", "CFB", "GCM"]:
    start_time = time.time()
    plaintext = "Data rahasia untuk eksperimen yang lebih kompleks."
    ciphertext = encrypt_aes(plaintext, key, mode)
    end_time = time.time()
    print(f"Mode {mode} - Waktu Enkripsi: {end_time - start_time:.6f} detik")
    print(f"Ciphertext ({mode}): {ciphertext.hex()[:50]}...\n")

Mode CBC - Waktu Enkripsi: 0.000000 detik
Ciphertext (CBC): f8e1bf3b59bd6236966c33d421c26bd67439cdefba01f6d640...

Mode CFB - Waktu Enkripsi: 0.005996 detik
Ciphertext (CFB): 9148ebd31960fcb2ee2acd22a8299f5423fb7af3c91d3813b2...

Mode GCM - Waktu Enkripsi: 0.001001 detik
Ciphertext (GCM): 082d766fa1c3a39c2a991ca289ff5069dbe584ba15ffd65220...



6. Apa kegunaan dan perbedaan masing-masing mode AES ini?

In [11]:
from itertools import product
import string

# Fungsi brute force terhadap kunci pendek
def brute_force_attack(target_ciphertext, correct_key):
    charset = string.ascii_letters + string.digits
    key_length = len(correct_key)
    print(f"Memulai brute force pada kunci {key_length} karakter...")

    for attempt in product(charset, repeat=key_length):
        guessed_key = "".join(attempt).encode()
        if guessed_key == correct_key:
            print(f"Kunci ditemukan: {guessed_key.decode()}")
            break

# Simulasi kunci pendek (4 karakter)
correct_key = b"abc1"

ciphertext = encrypt_aes("Tes brute force", correct_key.ljust(32, b'\x00'), "CBC")

# Menjalankan brute force
brute_force_attack(ciphertext, correct_key)


Memulai brute force pada kunci 4 karakter...
Kunci ditemukan: abc1


7. Cobalah berbagai variasi key dan key yang lebih panjang, amati prilakunya dan jelaskan mengapa AES direkomendasikan menggunakan panjang key 256 bit

In [23]:
# Fungsi Enkripsi AES-GCM
def encrypt_gcm(plaintext, key):
    iv = os.urandom(12)  # IV 12-byte untuk GCM
    cipher = Cipher(algorithms.AES(key), modes.GCM(iv), backend=default_backend())
    encryptor = cipher.encryptor()

    ciphertext = encryptor.update(plaintext.encode()) + encryptor.finalize()
    tag = encryptor.tag  # Authentication tag untuk verifikasi

    return iv, ciphertext, tag  # Mengembalikan IV, Ciphertext, dan Tag

# Fungsi Dekripsi AES-GCM
def decrypt_gcm(iv, ciphertext, tag, key):
    cipher = Cipher(algorithms.AES(key), modes.GCM(iv, tag), backend=default_backend())
    decryptor = cipher.decryptor()

    try:
        decrypted_text = decryptor.update(ciphertext) + decryptor.finalize()
        #print(f"decript:{decrypted_text.hex()}")
        return decrypted_text.decode()
    except Exception as e:
        return f"[GCM DETECTED CHANGE] Dekripsi gagal: {str(e)}"

# Buat kunci AES-256 (32 byte)
key = os.urandom(32)
plaintext = "Pesan rahasia yang sangat penting"

# Enkripsi dengan AES-GCM
iv, ciphertext, tag = encrypt_gcm(plaintext, key)
print(f"Ciphertext (GCM): {ciphertext.hex()}\n")

# Dekripsi Tanpa Perubahan
decrypted_text = decrypt_gcm(iv, ciphertext, tag, key)
print(f" Dekripsi Sukses: {decrypted_text}\n")

# Mengubah Satu Byte di Ciphertext untuk Simulasi Serangan
ciphertext_modified = bytearray(ciphertext)
ciphertext_modified[5] ^= 0xFF  # Membalikkan satu byte pada index ke-5

# Coba Dekripsi dengan Ciphertext yang Telah Diubah
decrypted_text_modified = decrypt_gcm(iv, bytes(ciphertext_modified), tag, key)
print(f" Dekripsi Gagal Setelah Perubahan: {decrypted_text_modified}\n")


Ciphertext (GCM): 2cacc3c111d768d9ba6809e46c9c9ce3672563ed93e02aa5bba7b01b09f1bde1c6

 Dekripsi Sukses: Pesan rahasia yang sangat penting

 Dekripsi Gagal Setelah Perubahan: [GCM DETECTED CHANGE] Dekripsi gagal: 



In [27]:
# Fungsi Enkripsi AES-CBC dengan Padding PKCS7
def encrypt_cbc(plaintext, key, iv):
    cipher = Cipher(algorithms.AES(key), modes.CBC(iv), backend=default_backend())
    encryptor = cipher.encryptor()

    # Gunakan padding PKCS7 agar panjang plaintext menjadi kelipatan 16 byte
    padder = padding.PKCS7(128).padder()
    padded_plaintext = padder.update(plaintext.encode()) + padder.finalize()

    ciphertext = encryptor.update(padded_plaintext) + encryptor.finalize()
    return ciphertext

# Fungsi Dekripsi AES-CBC dengan Unpadding PKCS7
def decrypt_cbc(iv, ciphertext, key):
    cipher = Cipher(algorithms.AES(key), modes.CBC(iv), backend=default_backend())
    decryptor = cipher.decryptor()
    
    try:
        decrypted_padded = decryptor.update(ciphertext) + decryptor.finalize()
        print(f"decript:{decrypted_padded.hex()}")

        # Hapus padding setelah dekripsi
        unpadder = padding.PKCS7(128).unpadder()
        decrypted_text = unpadder.update(decrypted_padded) + unpadder.finalize()
        
        return decrypted_text.decode()
    
    except ValueError as e:
        return f"[CBC DETECTED ERROR] Dekripsi gagal: {str(e)}"

# Enkripsi dengan AES-CBC
key = os.urandom(32)  # AES-256 key
iv_cbc = os.urandom(16)  # IV untuk CBC
plaintext = "Pesan penting yang harus dirahasiakan"

ciphertext_cbc = encrypt_cbc(plaintext, key, iv_cbc)
print(f"Ciphertext (CBC): {ciphertext_cbc.hex()}\n")

# Modifikasi Satu Byte dalam Ciphertext (Simulasi Serangan)
ciphertext_cbc_modified = bytearray(ciphertext_cbc)

ciphertext_cbc_modified[5] ^= 0xFF  # Mengubah satu byte

# Dekripsi Ciphertext yang Telah Diubah
decrypted_cbc_modified = decrypt_cbc(iv_cbc, bytes(ciphertext_cbc_modified), key)
print(f" CBC Dekripsi Setelah Perubahan: {decrypted_cbc_modified}\n")


Ciphertext (CBC): 493f9c6880b161dad1d26a4c7d52993775a1f118460f2f7b8fa60af8c67154769b281c148c10e4f032a1ef5b775c3553

decript:ae09fdeb7b24998ef8ba9dcc7b61ac6d6e672068618d7573206469726168617369616b616e0b0b0b0b0b0b0b0b0b0b0b
 CBC Dekripsi Setelah Perubahan: [CBC DETECTED ERROR] Dekripsi gagal: 'utf-8' codec can't decode byte 0xae in position 0: invalid start byte



8. Bagaimana perbedaan respon model CBC dan mode GCM terhadap serangan?